In [4]:
import numpy as np

from ifqi import envs
from ifqi.evaluation import evaluation

In [5]:
mdp = envs.LQG1D()
state_dim, action_dim, reward_dim = envs.get_space_info(mdp)
actions = np.linspace(-8, 8, 100)

In [6]:
dataset = evaluation.collect_episodes(mdp, n_episodes=2000)

In [7]:
mdp.horizon

100

In [8]:
dataset.shape

(200000, 6)

In [9]:
winning_states = list(filter(lambda data: data[4] != 0, dataset))

In [10]:
winning_states

[]

In [11]:
class PBO:
    
    def __init__(self, theta, actions, training_set, gamma=1):
        self.theta = theta
        self.actions = actions
        self.training_set = training_set
        self.gamma = gamma
        
    def fitness(self, ro):
        return sum((self.Q(s, a, f_theta) - r - self.gamma *
                    max(self.Q(ns, a, theta) for a in self.actions) ) ** 2
                   for (s, a, r, ns, *_), (theta, f_theta)
                   in zip(self.training_set, self.theta_seq(ro)))
            
    def Q(self, s, a, theta):
        k, b = theta
        return b - (a - k * s) ** 2
    
    def theta_seq(self, ro):
        theta = self.theta
        self.theta = self.f(ro, theta)
        yield theta, self.theta
        
    def f(self, ro, theta):
        return ro.T * theta


In [12]:
pbo = PBO(np.array([1, 0]), actions, dataset)
pbo.fitness(np.array([1, 1]))

144.73740470004282

In [13]:
from pybrain.optimization import ExactNES

ro_i = np.array([1, 1])
optimizer = ExactNES(pbo.fitness, ro_i, minimize=True)

/home/tyrion/.local/lib/python3.5/site-packages/pybrain/optimization/distributionbased/ves.py:83: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  self.phiSquareWindow = zeros((self.batchSize, self.numDistrParams))


In [16]:
ro, score = optimizer.learn()
ro, score

/home/tyrion/.local/lib/python3.5/site-packages/pybrain/optimization/distributionbased/ves.py:89: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if z == None:
/home/tyrion/.local/lib/python3.5/site-packages/ipykernel/__main__.py:12: RuntimeWarning: overflow encountered in double_scalars
[2016-11-20 18:34:31,276] Algorithm diverged. Stopped after 305 learning steps.


(array([-1.86597057, -1.73737723]), 0.077457038620063201)